In [1]:
!pip show keras

Name: keras
Version: 3.2.1
Summary: Multi-backend Keras.
Home-page: https://github.com/keras-team/keras
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache License 2.0
Location: /opt/conda/lib/python3.10/site-packages
Requires: absl-py, h5py, ml-dtypes, namex, numpy, optree, rich
Required-by: keras-tuner, tensorflow
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import json
import cv2
import numpy as np

In [2]:
img_width, img_height = 96, 96
# Рgrayscale image 148x148
# backend Tensorflow => channels_last
input_shape = (img_width, img_height, 1)

dataset_folder = '/kaggle/input/afew-va-cropped/AFEW_VA'

# Initialize lists to store images and labels
images = []
valence_labels = []
arousal_labels = []

nfd = 1
for folders in os.listdir(dataset_folder):
  folder_path_ = os.path.join(dataset_folder, folders)
  #nfd = nfd+1
  # Iterate through each video folder
  for folder_name in range(1, 601):
      #nfd+=1
      folder_name_str = str(folder_name).zfill(3)  # Zero padding for folder names (001, 002, ..., 600)
      folder_path = os.path.join(folder_path_, folder_name_str)
      folder_path_cropped = os.path.join(folder_path_,folder_name_str+str("_cropped"))
      print(folder_path)
      if os.path.isdir(folder_path) and os.path.isdir(folder_path_cropped):
          #j = -1
          # Load JSON file containing annotations
          json_file = os.path.join(folder_path, folder_name_str + '.json')
          with open(json_file) as f:
              data = json.load(f)
          print(json_file)
          # Iterate through frames in the video
          for frame_id, frame_data in data['frames'].items():
#               j = j+1
#               if j%3 !=0 :
#                 continue
              #print(frame_id)
              # Read image
              image_file = os.path.join(folder_path_cropped, frame_id + '.png')
              if not os.path.isfile(image_file):
                continue
              print(image_file)
              #image = cv2.imread(image_file, cv2.COLOR_BGR2GRAY) #IMREAD_GRAYSCALE)  # Read as grayscale
              image = cv2.imread(image_file)
              try:
                image = cv2.resize(image, (img_width, img_height))
              except Exception as e:
                print(f"Error: Resized image is empty {image_file}")
                continue
              image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
              # Extract valence and arousal labels
              valence = frame_data['valence']
              arousal = frame_data['arousal']

              # Append image and labels to lists
              images.append(image)
              valence_labels.append(valence)
              arousal_labels.append(arousal)

  nfd+=1

# Convert lists to numpy arrays
x_data = np.array(images)
y_valence = np.array(valence_labels)
y_arousal = np.array(arousal_labels)

print(x_data.shape,y_valence.shape,y_arousal.shape)

/kaggle/input/afew-va-cropped/AFEW_VA/IP12/001
/kaggle/input/afew-va-cropped/AFEW_VA/IP12/002
/kaggle/input/afew-va-cropped/AFEW_VA/IP12/003
/kaggle/input/afew-va-cropped/AFEW_VA/IP12/004
/kaggle/input/afew-va-cropped/AFEW_VA/IP12/005
/kaggle/input/afew-va-cropped/AFEW_VA/IP12/006
/kaggle/input/afew-va-cropped/AFEW_VA/IP12/007
/kaggle/input/afew-va-cropped/AFEW_VA/IP12/008
/kaggle/input/afew-va-cropped/AFEW_VA/IP12/009
/kaggle/input/afew-va-cropped/AFEW_VA/IP12/010
/kaggle/input/afew-va-cropped/AFEW_VA/IP12/011
/kaggle/input/afew-va-cropped/AFEW_VA/IP12/012
/kaggle/input/afew-va-cropped/AFEW_VA/IP12/013
/kaggle/input/afew-va-cropped/AFEW_VA/IP12/014
/kaggle/input/afew-va-cropped/AFEW_VA/IP12/015
/kaggle/input/afew-va-cropped/AFEW_VA/IP12/016
/kaggle/input/afew-va-cropped/AFEW_VA/IP12/017
/kaggle/input/afew-va-cropped/AFEW_VA/IP12/018
/kaggle/input/afew-va-cropped/AFEW_VA/IP12/019
/kaggle/input/afew-va-cropped/AFEW_VA/IP12/020
/kaggle/input/afew-va-cropped/AFEW_VA/IP12/021
/kaggle/input

In [3]:
import os
import tensorflow
from PIL import ImageFile, Image
from keras.preprocessing.image import load_img
from keras.preprocessing.image import save_img
from keras.preprocessing.image import img_to_array
import glob
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout, BatchNormalization, LSTM, TimeDistributed, SpatialDropout1D
#from google.colab.patches import cv2_imshow

2024-05-08 07:43:03.364110: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-08 07:43:03.364230: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-08 07:43:03.478957: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# image size
img_width, img_height = 96, 96
# Рgrayscale image 148x148
# backend Tensorflow => channels_last
input_shape = (img_width, img_height, 1)

#ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
# filling object-feature matrix
X = []
filenames = []
c = 0
for filename in sorted(glob.glob('/kaggle/input/emodataset/Emodataset/train/*')):
  image = cv2.imread(filename)
  image = cv2.resize(image, (img_width, img_height))
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  filenames.append(filename)
  X.append(image)
  c += 1
  print(c)
print(X[0])

In [ ]:
# for filename in sorted(glob.glob('/content/drive/My Drive/train/*.jpg')):
#   image = cv2.imread(filename)
#   image = cv2.resize(image, (img_width, img_height))
#   image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#   print(cv2_imshow(image))

In [ ]:
# setting pixels values to range [0;1]
X = np.divide(X, 255.0)

In [ ]:
# reading file with answers for images
df = pd.read_csv('/kaggle/input/emodataset/Emodataset/train_labels.csv', sep=',')
df = df[['pth', 'valence', 'arousal']]
#Y['pth'] = Y['pth'].apply(lambda x: x.split("/")[1])
df = df.sort_values('pth')

In [ ]:
df.head()

In [ ]:
# Y1 = []
# for index, row in Y.iterrows():
#   for f in filenames:
#     if f.find(row['subDirectory_filePath']) >= 0:
#       Y1.append(row)
#       print("appending " + str(row))
#       break

In [ ]:
Y = pd.read_csv('/kaggle/input/emodataset/Emodataset/test_labels.csv', sep=',')
Y = Y[['pth', 'valence', 'arousal']]
#df['pth'] = df['pth'].apply(lambda x: x.split("/")[1])
Y

In [ ]:
# concatenation filenames with X matrix
d = dict(zip(filenames, X))
list(d.keys())[0]

In [ ]:
len(d)

In [ ]:
files = df['pth']
new_dict = {}
for key in d.keys():
  for file in files:
    if key.find(file) >= 0:
      new_dict.update({key:d[key]})
      break
print(len(new_dict))

In [ ]:
# Filter the keys of new_dict based on the filenames in files
#filtered_dict = {key: new_dict[key] for key in new_dict.keys() if key in files}

# Assign the values of filtered_dict to the 'image' column in df
#df['image'] = list(filtered_dict.values())

df['image'] = list(new_dict.values())

In [ ]:
indexes_to_drop = df[(df['valence'] == -2) | (df['arousal'] == -2)].index
df.drop(indexes_to_drop, inplace=True)

In [30]:
# def create_model2():
#     # Define input layer
#     inputs = Input(shape=(img_width, img_height, 1))

#     # Define common model architecture
#     x = Conv2D(32, (3, 3), input_shape=(img_width, img_height, 1))(inputs)
#     x = Conv2D(32, (3, 3), activation='relu')(x)
#     x = MaxPooling2D(pool_size=(2, 2))(x)
#     x = Dropout(0.25)(x)
#     x = Conv2D(64, (3, 3), activation='relu')(x)
#     x = Conv2D(64, (3, 3), activation='relu')(x)
#     x = MaxPooling2D(pool_size=(2, 2))(x)
#     x = Dropout(0.25)(x)
#     x = Conv2D(128, (3, 3), activation='relu')(x)
#     x = Conv2D(128, (3, 3), activation='relu')(x)
#     x = MaxPooling2D(pool_size=(2, 2))(x)
#     x = Dropout(0.25)(x)
#     x = Conv2D(256, (3, 3), activation='relu')(x)
#     x = Conv2D(256, (3, 3), activation='relu')(x)
#     x = MaxPooling2D(pool_size=(2, 2))(x)
#     x = Dropout(0.25)(x)
#     x = Flatten()(x)
#     x = Dense(150, activation='relu')(x)
#     x = Dropout(0.5)(x)

#     # Define output layers for valence and arousal
#     output_valence = Dense(1, activation='linear', name='valence_output')(x)
#     output_arousal = Dense(1, activation='linear', name='arousal_output')(x)

#     # Define model with inputs and outputs
#     model = Model(inputs=inputs, outputs=[output_valence, output_arousal])

#     return model

In [5]:
from keras.layers import Input

def create_model():
    inputs = Input(shape=(img_width, img_height, 1))
    # common model for both valence and arousal
    model = Sequential()
    model.add(inputs)
    model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 1)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(150, activation='relu'))
    model.add(Dropout(0.5))
    #print(model.layers[-1].output.shape)
    #model.add(Dense(2, activation='linear'))
    # Output layer for valence 
    valence_output = Dense(1, activation='linear', name='valence_output')(model.layers[-1].output)
    # Output layer for arousal
    arousal_output = Dense(1, activation='linear', name='arousal_output')(model.layers[-1].output)

    # Combine outputs
    model = Model(inputs=inputs, outputs=[valence_output, arousal_output])
    
    return model

#model.summary()

#model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['accuracy'])

In [6]:
from tensorflow.keras.optimizers import SGD

# Define and compile model for valence training
model_va = create_model()  # Using the same architecture for valence
model_va.summary()
#model_va.compile(loss={'valence_output': 'mse', 'arousal_output': 'mse'}, optimizer='sgd')
model_va.compile(loss=['mse', 'mse'], optimizer='sgd')

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "functional_20"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 96, 96, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 94, 94,    │        320 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 92, 92,    │      9,248 │ conv2d[18][0]     │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 46, 46,    │          0 │ conv2d_1[17][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 46, 46,    │          0 │ max_pooling2d[16… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 44, 44,    │     18,496 │ dropout[15][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 42, 42,    │     36,928 │ conv2d_2[14][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 21, 21,    │          0 │ conv2d_3[13][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 21, 21,    │          0 │ max_pooling2d_1[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 19, 19,    │     73,856 │ dropout_1[11][0]  │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 17, 17,    │    147,584 │ conv2d_4[10][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 8, 8, 128) │          0 │ conv2d_5[9][0]    │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 8, 8, 128) │          0 │ max_pooling2d_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 6, 6, 256) │    295,168 │ dropout_2[7][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 4, 4, 256) │    590,080 │ conv2d_6[6][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 2, 2, 256) │          0 │ conv2d_7[5][0]    │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 2, 2, 256) │          0 │ max_pooling2d_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 1024)      │          0 │ dropout_3[3][0]   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 1,325,732 (5.06 MB)

 Trainable params: 1,325,732 (5.06 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from keras.optimizers import SGD
from keras.losses import MeanSquaredError

optimizer = SGD(learning_rate=0.01, momentum=0.9)

In [ ]:
# Define and compile model for valence training
model_v = create_model()  # Using the same architecture for valence
model_v.summary()
model_v.compile(loss='mean_squared_error', optimizer='sgd')

In [ ]:
# Define and compile model for arousal training
model_a = create_model()  # Using the same architecture for arousal
model_a.compile(loss='mean_squared_error', optimizer='sgd')

In [ ]:
# Define and compile model for valence training
model_va = create_model()  # Using the same architecture for valence
model_va.summary()
model_va.compile(loss='mean_squared_error', optimizer='sgd')

In [7]:
# Convert each image to grayscale
#x_data_gray = np.array([cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) for image in x_data])
x_data.shape

(29209, 96, 96)

In [8]:
x_data = np.divide(x_data, 255.0)

In [9]:
#valence arousal training

# Combine valence and arousal labels into a single array
y_combined = np.hstack((y_valence.reshape(-1, 1), y_arousal.reshape(-1, 1)))

# Split the combined data and labels into training, validation, and test sets
x_train_va, x_valid_va, y_train_va, y_valid_va = train_test_split(x_data, y_combined, test_size=0.33)
x_valid_va, x_test_va, y_valid_va, y_test_va = train_test_split(x_valid_va, y_valid_va, test_size=0.5)

# Split the combined train labels into separate valence and arousal labels
y_train_v = y_train_va[:, 0]  # Valence labels
y_train_a = y_train_va[:, 1]  # Arousal labels

# Split the combined validation labels into separate valence and arousal labels
y_valid_v = y_valid_va[:, 0]  # Valence labels
y_valid_a = y_valid_va[:, 1]  # Arousal labels

# Split the combined test labels into separate valence and arousal labels
y_test_v = y_test_va[:, 0]  # Valence labels
y_test_a = y_test_va[:, 1]  # Arousal labels


# x_train_v, x_valid_v, y_train_v, y_valid_v = train_test_split(x_data, y_valence, test_size=0.33)

# # Split validation set into half to get separate validation and test sets
# x_valid_v, x_test_v, y_valid_v, y_test_v = train_test_split(x_valid_v, y_valid_v, test_size=0.5)

#print(x_train_v.shape)
x_train_va = x_train_va.reshape(x_train_va.shape[0], img_width, img_height, 1)
x_valid_va = x_valid_va.reshape(x_valid_va.shape[0], img_width, img_height, 1)
x_test_va = x_test_va.reshape(x_test_va.shape[0], img_width, img_height, 1)

import numpy as np

# Convert target valence labels to arrays
y_train_v = np.array(y_train_v)
y_valid_v = np.array(y_valid_v)
y_test_v = np.array(y_test_v)

y_train_v = y_train_v.reshape(y_train_v.shape[0],1)
y_valid_v = y_valid_v.reshape(y_valid_v.shape[0],1)
y_test_v = y_test_v.reshape(y_test_v.shape[0],1)

# Convert target arousal labels to arrays
y_train_a = np.array(y_train_a)
y_valid_a = np.array(y_valid_a)
y_test_a = np.array(y_test_a)

y_train_a = y_train_a.reshape(y_train_a.shape[0],1)
y_valid_a = y_valid_a.reshape(y_valid_a.shape[0],1)
y_test_a = y_test_a.reshape(y_test_a.shape[0],1)

# Train model for valence arousal
# from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau

# # Reduce learning rate on plateau callback
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.0001, verbose=1)

# # Early stopping callback (monitor validation loss)
# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

#model_va.fit(x_train_va, {'valence_output': y_train_v, 'arousal_output': y_train_a}, epochs=2, batch_size=64, validation_data=(x_valid_va, {'valence_output': y_valid_v, 'arousal_output': y_valid_a}))
history = model_va.fit(x_train_va, [y_train_v, y_train_a], epochs=100, batch_size=64, validation_data=(x_valid_va, [y_valid_v, y_valid_a]), callbacks=[])

#training_history = history.history

#print(training_history)
# # Get individual loss lists for valence and arousal (assuming key names)
# val_loss_v = training_history['val_loss'][0]  # Assuming key name for valence loss
# val_loss_a = training_history['val_loss'][1]  # Assuming key name for arousal loss

# print("valence val_loss:", val_loss_v," arousal val_loss:", val_loss_a)  # Access last epoch's loss
# model_va.fit(x_train_va, [y_train_v, y_train_a],
#                     validation_data=(x_valid_va, [y_valid_v, y_valid_a]), 
#                     epochs=100, batch_size=64)
#model_v.fit(x_train_v, y_train_v, epochs=100, batch_size=64, validation_data=(x_valid_v, y_valid_v))

Epoch 1/100
  2/306 ━━━━━━━━━━━━━━━━━━━━ 18s 60ms/step - loss: 23.2192   

I0000 00:00:1715154943.321092     143 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1715154943.343822     143 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


306/306 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 15.9632

W0000 00:00:1715154968.908676     144 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1715154970.012249     145 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


306/306 ━━━━━━━━━━━━━━━━━━━━ 47s 96ms/step - loss: 15.9593 - val_loss: 14.5653
Epoch 2/100
  3/306 ━━━━━━━━━━━━━━━━━━━━ 11s 38ms/step - loss: 14.2043

W0000 00:00:1715154972.814100     146 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


306/306 ━━━━━━━━━━━━━━━━━━━━ 12s 41ms/step - loss: 14.2541 - val_loss: 14.5441
Epoch 3/100
306/306 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - loss: 14.2637 - val_loss: 14.5217
Epoch 4/100
306/306 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - loss: 14.3433 - val_loss: 14.5281
Epoch 5/100
306/306 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - loss: 14.1037 - val_loss: 14.5172
Epoch 6/100
306/306 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - loss: 14.0401 - val_loss: 14.5187
Epoch 7/100
306/306 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - loss: 14.2751 - val_loss: 14.5094
Epoch 8/100
306/306 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - loss: 14.3622 - val_loss: 14.4602
Epoch 9/100
306/306 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - loss: 14.2380 - val_loss: 14.3649
Epoch 10/100
306/306 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - loss: 14.1195 - val_loss: 14.5139
Epoch 11/100
306/306 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - loss: 13.9553 - val_loss: 13.8727
Epoch 12/100
306/306 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - loss: 13.9897 - val_loss: 14.2110
Epoch 13

In [68]:
model_va.fit(x_train_va, [y_train_v, y_train_a], epochs=20, batch_size=64, validation_data=(x_valid_va, [y_valid_v, y_valid_a]))

Epoch 1/20
  3/306 ━━━━━━━━━━━━━━━━━━━━ 16s 56ms/step - loss: 0.9494

W0000 00:00:1715158550.411430     145 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


306/306 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.8641

W0000 00:00:1715158568.712193     144 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1715158569.742153     144 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


306/306 ━━━━━━━━━━━━━━━━━━━━ 27s 67ms/step - loss: 0.8640 - val_loss: 0.7950
Epoch 2/20
  3/306 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - loss: 0.7285

W0000 00:00:1715158570.868258     145 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


306/306 ━━━━━━━━━━━━━━━━━━━━ 12s 41ms/step - loss: 0.8353 - val_loss: 0.8209
Epoch 3/20
306/306 ━━━━━━━━━━━━━━━━━━━━ 12s 41ms/step - loss: 0.8416 - val_loss: 0.7892
Epoch 4/20
306/306 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - loss: 0.8307 - val_loss: 0.9695
Epoch 5/20
306/306 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - loss: 0.8481 - val_loss: 1.2131
Epoch 6/20
306/306 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - loss: 0.8282 - val_loss: 0.7873
Epoch 7/20
306/306 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - loss: 0.8186 - val_loss: 1.0198
Epoch 8/20
306/306 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - loss: 0.8104 - val_loss: 0.8518
Epoch 9/20
306/306 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - loss: 0.8463 - val_loss: 0.8267
Epoch 10/20
306/306 ━━━━━━━━━━━━━━━━━━━━ 12s 41ms/step - loss: 0.8211 - val_loss: 0.9110
Epoch 11/20
306/306 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - loss: 0.8283 - val_loss: 0.8774
Epoch 12/20
306/306 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - loss: 0.8032 - val_loss: 0.9480
Epoch 13/20
306/306 ━━━━━━━━━━━━━━━━━━━━

In [10]:
#scores = model_va.evaluate(x_test_va, [y_test_v,y_test_a],verbose=2)
#scores
print("score = " + str(model_va.evaluate(x_test_va, [y_test_v,y_test_a])))
#print("valence score = " + str(scores[0]))
#print("arousal score = " + str(scores[1]))

 29/151 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.7513

W0000 00:00:1715156233.011337     143 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


151/151 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 0.7462
score = 0.7638255953788757


In [11]:
print(x_test_va.shape)
x_sample = x_test_va[1000].reshape(1, img_width, img_height, 1)
model_va.predict(x_sample),[y_test_v[1000],y_test_a[1000]]

(4820, 96, 96, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


([array([[-2.4379158]], dtype=float32), array([[3.0478973]], dtype=float32)],
 [array([-2.]), array([3.])])

In [67]:
x_sample = x_test_va[210].reshape(1, img_width, img_height, 1)
loaded_model_va.predict(x_sample),[y_test_v[210],y_test_a[210]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


([array([[-1.6764443]], dtype=float32), array([[2.9240546]], dtype=float32)],
 [array([-2.]), array([3.])])

In [62]:
for i in range(y_test_v.shape[0]):
    if(y_test_v[i]<0 and y_test_a[i]>0):
        print(i)

3
9
10
12
14
17
19
23
25
27
28
31
32
33
34
35
36
38
39
40
41
42
47
51
54
57
59
61
62
64
66
67
70
73
76
77
78
82
84
85
86
94
95
96
99
100
101
103
110
111
115
117
118
120
121
122
125
126
127
128
129
131
134
139
140
141
142
144
148
150
151
152
155
156
158
161
162
165
171
172
176
178
180
181
182
183
184
187
188
190
192
194
195
197
201
207
210
216
222
223
224
226
227
228
230
232
234
235
238
241
244
245
247
249
250
252
253
255
258
260
261
265
266
267
268
276
277
282
289
291
292
295
298
303
304
306
309
310
314
318
319
320
321
324
326
327
328
330
333
336
338
340
341
343
345
347
348
350
355
357
358
360
362
364
365
366
367
368
370
374
375
378
380
381
382
386
388
389
391
392
395
396
400
401
403
405
406
411
418
423
425
426
427
434
435
437
440
441
443
449
450
451
453
454
459
460
463
464
466
472
474
477
480
484
489
490
495
496
503
504
510
512
513
515
519
520
521
522
523
527
532
533
535
536
544
549
552
555
562
564
566
567
571
572
581
582
585
589
590
595
598
599
604
605
606
607
608
609
612
616
617
624

In [12]:
y_pred_v, y_pred_a = model_va.predict(x_test_va)
#len(y_pred_va)

 24/151 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step

W0000 00:00:1715156265.772319     144 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


151/151 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step


In [24]:
#Evaluation metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
print('Performance Evaluation\n')
#Valence
mse = mean_squared_error(y_test_v, y_pred_v)
mae = mean_absolute_error(y_test_v, y_pred_v)
rmse = np.sqrt(mean_squared_error(y_test_v, y_pred_v))
r2 = r2_score(y_test_v, y_pred_v)
evs = explained_variance_score(y_test_v, y_pred_v)
print('Valence')
print('MSE: ',mse)
print('MAE: ',mae)
print('RMSE: ',rmse)
print('R2 score: ',r2)
print('EVS: ',evs)
#Arousal
mse = mean_squared_error(y_test_a, y_pred_a)
mae = mean_absolute_error(y_test_a, y_pred_a)
rmse = np.sqrt(mean_squared_error(y_test_a, y_pred_a))
r2 = r2_score(y_test_a, y_pred_a)
evs = explained_variance_score(y_test_a, y_pred_a)
print('\nArousal')
print('MSE: ',mse)
print('MAE: ',mae)
print('RMSE: ',rmse)
print('R2 score: ',r2)
print('EVS: ',evs)

Performance Evaluation

Valence
MSE:  0.3976867241050386
MAE:  0.418251515974395
RMSE:  0.63062407510738
R2 score:  0.9475542776826267
EVS:  0.9476625961992398

Arousal
MSE:  0.36613891291103834
MAE:  0.41433957738500415
RMSE:  0.605094135578125
R2 score:  0.9437413241776456
EVS:  0.9441631580856823


In [30]:
model_va.compile(loss=['mean_squared_error','mean_squared_error'], optimizer='sgd')
model_va.save('/kaggle/working/vamodel.h5')

In [32]:
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import mse

# Assuming your saved model filepath is 'my_model.h5'
loaded_model_va = load_model('/kaggle/working/vamodel.h5')
#loaded_model_va = load_model('/kaggle/working/vamodel.h5')
loaded_model_va.compile(loss=['mse','mse'], optimizer='sgd')

In [ ]:
y_test_a.shape

In [ ]:
x_train_v.shape,y_train_v.shape

In [ ]:
y_train_v[100]

In [ ]:
x_train_v[0]

In [ ]:
y_train_v

In [ ]:
y_valence_ = np.divide(y_valence, 10.0)

In [ ]:
y_valence_

In [ ]:
# valence training
x_train_v, x_valid_v, y_train_v, y_valid_v = train_test_split(x_data, y_valence, test_size=0.33)

# Split validation set into half to get separate validation and test sets
x_valid_v, x_test_v, y_valid_v, y_test_v = train_test_split(x_valid_v, y_valid_v, test_size=0.5)

#print(x_train_v.shape)
x_train_v = x_train_v.reshape(x_train_v.shape[0], img_width, img_height, 1)
x_valid_v = x_valid_v.reshape(x_valid_v.shape[0], img_width, img_height, 1)
x_test_v = x_test_v.reshape(x_test_v.shape[0], img_width, img_height, 1)

import numpy as np

# Convert target labels to arrays
y_train_v = np.array(y_train_v)
y_valid_v = np.array(y_valid_v)
y_test_v = np.array(y_test_v)

y_train_v = y_train_v.reshape(y_train_v.shape[0],1)
y_valid_v = y_valid_v.reshape(y_valid_v.shape[0],1)
y_test_v = y_test_v.reshape(y_test_v.shape[0],1)

# Train model for valence
model_v.fit(x_train_v, y_train_v, epochs=100, batch_size=64, validation_data=(x_valid_v, y_valid_v))

In [ ]:
model_v.fit(x_train_v, y_train_v, epochs=50, batch_size=64, validation_data=(x_valid_v, y_valid_v))

In [ ]:
# arousal training
x_train_a, x_valid_a, y_train_a, y_valid_a = train_test_split(x_data, y_arousal, test_size=0.33)

# Split validation set into half to get separate validation and test sets
x_valid_a, x_test_a, y_valid_a, y_test_a = train_test_split(x_valid_a, y_valid_a, test_size=0.5)

#print(x_train_v.shape)
x_train_a = x_train_a.reshape(x_train_a.shape[0], img_width, img_height, 1)
x_valid_a = x_valid_a.reshape(x_valid_a.shape[0], img_width, img_height, 1)
x_test_a = x_test_a.reshape(x_test_a.shape[0], img_width, img_height, 1)

import numpy as np

# Convert target labels to arrays
y_train_a = np.array(y_train_a)
y_valid_a = np.array(y_valid_a)
y_test_a = np.array(y_test_a)

y_train_a = y_train_a.reshape(y_train_a.shape[0],1)
y_valid_a = y_valid_a.reshape(y_valid_a.shape[0],1)
y_test_a = y_test_a.reshape(y_test_a.shape[0],1)

# Train model for valence
model_a.fit(x_train_a, y_train_a, epochs=100, batch_size=64, validation_data=(x_valid_a, y_valid_a))

In [ ]:
# arousal training
x_train_a, x_valid_a, y_train_a, y_valid_a = train_test_split(x_data, y_arousal, test_size=0.33)

# Split validation set into half to get separate validation and test sets
x_valid_a, x_test_a, y_valid_a, y_test_a = train_test_split(x_valid_a, y_valid_a, test_size=0.5)

#print(x_train_v.shape)
x_train_a = x_train_a.reshape(x_train_a.shape[0], img_width, img_height, 1)
x_valid_a = x_valid_a.reshape(x_valid_a.shape[0], img_width, img_height, 1)
x_test_a = x_test_a.reshape(x_test_a.shape[0], img_width, img_height, 1)

import numpy as np

# Convert target labels to arrays
y_train_a = np.array(y_train_a)
y_valid_a = np.array(y_valid_a)
y_test_a = np.array(y_test_a)

y_train_a = y_train_a.reshape(y_train_a.shape[0],1)
y_valid_a = y_valid_a.reshape(y_valid_a.shape[0],1)
y_test_a = y_test_a.reshape(y_test_a.shape[0],1)


In [ ]:
model_a.fit(x_train_a, y_train_a, epochs=50, batch_size=64, validation_data=(x_valid_a, y_valid_a))

In [ ]:
model_a.fit(x_train_a, y_train_a, epochs=20, batch_size=64, validation_data=(x_valid_a, y_valid_a))

In [ ]:
df['arousal'],df

In [ ]:
print("score = " + str(model_v.evaluate(x_test_v, y_test_v)))

In [ ]:
print("score = " + str(model_a.evaluate(x_test_a, y_test_a)))

In [ ]:
x_train_v[0].shape

In [ ]:
print(x_test_v.shape)
x_sample = x_test_v[1000].reshape(1, img_width, img_height, 1)
model_v.predict(x_sample),y_test_v[1000]

In [ ]:
x_sample = x_test_v[120].reshape(1, img_width, img_height, 1)
loaded_model_v.predict(x_sample),y_test_v[120]

In [ ]:
x_sample = x_test_a[100].reshape(1, img_width, img_height, 1)
model_a.predict(x_sample),y_test_a[100]

In [ ]:
x_sample = x_test_a[200].reshape(1, img_width, img_height, 1)
loaded_model_a.predict(x_sample),y_test_a[200]

In [ ]:
y_pred_v = model_v.predict(x_test_v)

In [ ]:
y_pred_a = model_a.predict(x_test_a)

In [ ]:
#Evaluation metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
#Valence
mse = mean_squared_error(y_test_v, y_pred_v)
mae = mean_absolute_error(y_test_v, y_pred_v)
rmse = np.sqrt(mean_squared_error(y_test_v, y_pred_v))
r2 = r2_score(y_test_v, y_pred_v)
evs = explained_variance_score(y_test_v, y_pred_v)
print('Valence model')
print('MSE: ',mse)
print('MAE: ',mae)
print('RMSE: ',rmse)
print('R2 score: ',r2)
print('EVS: ',evs)

In [ ]:
#Arousal
mse = mean_squared_error(y_test_a, y_pred_a)
mae = mean_absolute_error(y_test_a, y_pred_a)
rmse = np.sqrt(mean_squared_error(y_test_a, y_pred_a))
r2 = r2_score(y_test_a, y_pred_a)
evs = explained_variance_score(y_test_a, y_pred_a)
print('\nArousal model')
print('MSE: ',mse)
print('MAE: ',mae)
print('RMSE: ',rmse)
print('R2 score: ',r2)
print('EVS: ',evs)

In [ ]:
model_v.save('/kaggle/working/vmodel2.h5')

In [ ]:
from keras.models import load_model

loaded_model_v = load_model('/kaggle/working/vmodel2.h5')
loaded_model_v.compile(loss='mean_squared_error', optimizer='sgd')

In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()  # Provide your Kaggle credentials
api.dataset_download_file('/kaggle/', file_name='vmodel1.h5')

In [ ]:
model_a.save('/kaggle/working/amodel2.h5')

In [ ]:
from keras.models import load_model

loaded_model_a = load_model('/kaggle/working/amodel2.h5')
loaded_model_a.compile(loss='mean_squared_error', optimizer='sgd')

In [ ]:
#valence arousal estimation
import cv2
import numpy as np
from keras.models import load_model
from keras.preprocessing import image
import matplotlib.pyplot as plt

# Define the path to the image
img_path = "/content/ffhq_210.png" #"path/to/your/image"

try:
    img=plt.imread(img_path)
    plt.imshow(img)
    plt.axis('off')
    plt.show()
    img = image.load_img(img_path, target_size=(96, 96), color_mode='grayscale')
except Exception as e:
    print("Error loading image:", e)
    exit(1)



x = image.img_to_array(img)
x /= 255
x = np.expand_dims(x, axis=0)

loaded_model_v = load_model('/content/drive/MyDrive/Emonet/models/vmodel1.h5')
loaded_model_a = load_model('/content/drive/MyDrive/Emonet/models/amodel1.h5')

loaded_model_v.compile(loss='mean_squared_error', optimizer='sgd')
loaded_model_a.compile(loss='mean_squared_error', optimizer='sgd')

prediction_v = loaded_model_v.predict(x)
prediction_a = loaded_model_a.predict(x)

print('valence ',prediction_v[0][0],' arousal ',prediction_a[0][0])
# Determine emotion based on predictions
# if -0.5 <= prediction_v <= 0.5 and prediction_a <= -0.66:
#     print("focus")
# elif -0.2 <= prediction_v <= 0 and -0.66 <= prediction_a <= 0:
#     print("relaxation")
# elif prediction_v <= 0 and prediction_a <= 0:
#     print("stress")
# elif 0 <= prediction_v <= 0.4 and 0 <= prediction_a <= 0.4:
#     print("interest")
# elif (0 <= prediction_v <= 0.7 and 0.4 <= prediction_a <= 0.7 or
#       0.4 <= prediction_v <= 0.7 and 0 <= prediction_a <= 0.4 or
#       0.7 <= prediction_v <= 1 and 0 <= prediction_a <= 1):
#     print("engagement")
# elif 0 <= prediction_v <= 1 and 0.7 <= prediction_a <= 1:
#     print("excitement")
# else:
#     print("Cannot determine emotion")

In [ ]:
print("score = " + str(1 - model_a.evaluate(x_test_a, y_test_a)))

In [ ]:
#valence arousal estimation
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image

# Define the path to the image
img_path = "/kaggle/input/emodataset/Emodataset/train/ffhq_1.png" #"path/to/your/image"

try:
    img = image.load_img(img_path, target_size=(96, 96), color_mode='grayscale')
except Exception as e:
    print("Error loading image:", e)
    exit(1)

x = image.img_to_array(img)
x /= 255
x = np.expand_dims(x, axis=0)

#prediction_v = model_v.predict(x)
prediction_a = model_a.predict(x)

#print('valence ',prediction_v[0][0])
print(' arousal ',prediction_a[0][0])

In [ ]:
# saving valence model to file
model_json_v = model_v.to_json()

json_file = open("/kaggle/working/affect_model_v.json", "w")
json_file.write(model_json_v)
json_file.close()

model_v.save_weights("/kaggle/working/affect_model_v.weights.h5")

In [ ]:
#valence arousal estimation
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image

# Define the path to the image
img_path = "/kaggle/input/emodataset/Emodataset/test/ffhq_186.png" #"path/to/your/image"

try:
    img = image.load_img(img_path, target_size=(96, 96), color_mode='grayscale')
except Exception as e:
    print("Error loading image:", e)
    exit(1)

x = image.img_to_array(img)
x /= 255
x = np.expand_dims(x, axis=0)

json_file_v = open("/kaggle/working/affect_model_v.json", "r")
json_file_a = open("/kaggle/working/affect_model_a.json", "r")

loaded_model_json_v = json_file_v.read()
loaded_model_json_a = json_file_a.read()

json_file_v.close()
json_file_a.close()

loaded_model_v = model_from_json(loaded_model_json_v)
loaded_model_a = model_from_json(loaded_model_json_a)

loaded_model_v.load_weights("/kaggle/working/affect_model_v.weights.h5")
loaded_model_a.load_weights("/kaggle/working/affect_model_a.weights.h5")

loaded_model_v.compile(loss='mean_squared_error', optimizer='sgd')
loaded_model_a.compile(loss='mean_squared_error', optimizer='sgd')

prediction_v = loaded_model_v.predict(x)
prediction_a = loaded_model_a.predict(x)

print('valence ',prediction_v[0][0],' arousal ',prediction_a[0][0])
# Determine emotion based on predictions
# if -0.5 <= prediction_v <= 0.5 and prediction_a <= -0.66:
#     print("focus")
# elif -0.2 <= prediction_v <= 0 and -0.66 <= prediction_a <= 0:
#     print("relaxation")
# elif prediction_v <= 0 and prediction_a <= 0:
#     print("stress")
# elif 0 <= prediction_v <= 0.4 and 0 <= prediction_a <= 0.4:
#     print("interest")
# elif (0 <= prediction_v <= 0.7 and 0.4 <= prediction_a <= 0.7 or
#       0.4 <= prediction_v <= 0.7 and 0 <= prediction_a <= 0.4 or
#       0.7 <= prediction_v <= 1 and 0 <= prediction_a <= 1):
#     print("engagement")
# elif 0 <= prediction_v <= 1 and 0.7 <= prediction_a <= 1:
#     print("excitement")
# else:
#     print("Cannot determine emotion")


In [ ]:
# saving arousal model to file
model_json_a = model_a.to_json()

json_file = open("/kaggle/working/affect_model_a.json", "w")
json_file.write(model_json_a)
json_file.close()

model_a.save_weights("/kaggle/working/affect_model_a.weights.h5")